### Transaction
Redis transactions are created and executed using the `MULTI` and `EXEC` commands. Compared to transactions available in there are few differences: 

**Atomicity:** Redis transactions are atomic but not in the same sense as in RDBMS. In Redis, we first queue up sequence of operations and then execute all the operations in one go. If some error is detected while queueing up (before EXEC), none of the operation will get executed. However if all goes well and some problem occurs during execution (after EXEC), the commands that were executed successfully will not be rolled back.

We can see both the cases:

```
> MULTI
+OK
> SET greeting hello
+QUEUED
> LPOP greeting
+QUEUED
> EXEC
*2
+OK
-ERR Operation against a key holding the wrong kind of value
```

Error found after EXEC, the greeting key is set, this operation is not rolled back.

```
> MULTI
+OK
> SET greeting hello
+QUEUED
> INCR a b c
-ERR wrong number of arguments for 'incr' command
```

INCR command above is not queued at all

**Isolation:** Redis isolation level is serializable (when compared to RDBMS terminology), weaker isolation levels are not available.

Since the operations are all queued, we can't make decisions inside the transaction (which is why we use Lua scripts).

### Code Example

In [1]:
const redis = require('/usr/local/lib/node_modules/redis')
const client = redis.createClient()

In [2]:
let multi = client.multi()
multi.set('k1', '100')
multi.set('k2', '5')
multi.incr('k1')
multi.decr('k2')
// All the above commands have not executed yet
multi.exec((err, res)=>{
    for(r of res){
        console.log(r)
    }
})

true

OK
OK
101
4


### Pipeline
Pipeline in Redis differs from a transaction. It is primarily network optimization. It essentially means the client buffers up a bunch of commands and ships them to the server in one go. The commands are not guaranteed to be executed in a transaction. The benefit here is saving network round trip time for every command.

In [2]:
const pipeline = client.batch()
pipeline.hset('pipeline-test-key', 'available', 'false')
pipeline.expire('pipeline-test-key', 1000)
pipeline.exec((err, res)=>{
    for(r of res){
        console.log(r)
    }
})

true

0
1
